In [1]:
import os
import sys
sys.path.append(os.pardir)

In [2]:
import datetime
import pandas as pd
from sklearn import preprocessing
from credible import connectors

In [3]:
engine = connectors.connect_to_sqlite()

In [4]:
%%time
users = pd.read_sql_table('users', engine)
businesses = pd.read_sql_table('businesses', engine)
tips = pd.read_sql_table('tips', engine)
checkins = pd.read_sql_table('checkins', engine)
reviews = pd.read_sql_table('reviews', engine)
photos = pd.read_sql_table('photos', engine)

CPU times: user 1min 34s, sys: 47 s, total: 2min 21s
Wall time: 3min 54s


In [5]:
reviews.shape

(6685900, 10)

In [6]:
# derived columns to be inserted to database
df = pd.DataFrame()
df['review_id'] = reviews.review_id
df['business_id'] = reviews.business_id
df.head(2)

,review_id,business_id
0,Q1sbwvVQXV2734tPgoKj4Q,ujmEBvifdJM6h6RLv4wQIg
1,GJXCdrto3ASJOqKeVWPi6Q,NZnhc2sEQy3RmzKTZnqtwQ


## Get Information on Reviews

In [ ]:
reviews.stars.value_counts().plot.bar()

## Create Review Meta Information

### Review Length

In [31]:
df['text_length'] = reviews.text.apply(lambda x: len(x))

In [32]:
df['text_length_category'] = pd.qcut(df['text_length'], 10, labels=False)

In [33]:
df.head(2)

,review_id,business_id,date_date,newest_review_date,days_past,text_length,text_length_category
0,Q1sbwvVQXV2734tPgoKj4Q,ujmEBvifdJM6h6RLv4wQIg,2013-05-07 04:34:36,2018-11-14 06:12:10,2017,204,1
1,GJXCdrto3ASJOqKeVWPi6Q,NZnhc2sEQy3RmzKTZnqtwQ,2017-01-14 21:30:33,2018-11-03 15:13:54,657,1561,9


### Days Past

In [7]:
# conversion to datetime
df['date_date'] = pd.to_datetime(reviews.date)

In [10]:
# latest review date for each business
latest_review_for_business = df.groupby(
    'business_id', as_index=False).agg(
        {'review_id': 'count', 'date_date': 'max'})

In [23]:
# merge with df to get add the column
latest_review = pd.merge(
    df, latest_review_for_business, how='left', on='business_id')
df['newest_review_date'] = latest_review['date_date_y']

In [29]:
df['days_past'] = df.apply(
    lambda x: (x.newest_review_date - x.date_date).days, axis=1)

In [45]:
df = df.drop(['business_id', 'date_date'], axis=1)

In [46]:
df.head(2)

,review_id,newest_review_date,days_past,text_length,text_length_category
0,Q1sbwvVQXV2734tPgoKj4Q,2018-11-14 06:12:10,2017,204,1
1,GJXCdrto3ASJOqKeVWPi6Q,2018-11-03 15:13:54,657,1561,9


## Create the Table

In [47]:
df.to_sql(name='reviews_meta', con=engine, if_exists='replace', index=False)

In [48]:
pd.read_sql_table('reviews_meta', engine).head(2)

,review_id,newest_review_date,days_past,text_length,text_length_category
0,Q1sbwvVQXV2734tPgoKj4Q,2018-11-14 06:12:10,2017,204,1
1,GJXCdrto3ASJOqKeVWPi6Q,2018-11-03 15:13:54,657,1561,9
